<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/NN_ShaftTraectory_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
from numpy.core.memmap import uint8

from google.colab import drive

In [2]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#Создание датасета

In [4]:
# Советую воспользоваться циклом for, чтобы не писать множество однотипных строк кода

x = pd.read_excel(open('/content/drive/MyDrive/work/100.xlsx', 'rb'))
x = pd.DataFrame(x)
xnp = pd.DataFrame.to_numpy(x)
xnp = np.rot90(xnp)
xnp.shape

(6, 20000)

In [5]:
#parameters
num_samples = 10 # количество сэмплов в одном графике
point_number = 2000 # количество точек в одном графике
num_graphycs = 3 # количество графиков\
num_grahp0 = 1 # кол-во нормальных графиков
num_grahp1 = 2 # кол-во угловатых графиков
x = []

In [6]:
y0 = np.zeros((num_grahp0*num_samples,1))
y1 = np.ones((num_grahp1*num_samples,1))
y = np.concatenate((y0, y1))
y

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [7]:
for j in range (num_graphycs):
  for i in range (num_samples):
    x.append(np.concatenate((xnp[j*2,i*point_number:i*point_number+point_number], xnp[j*2+1,i*point_number:i*point_number+point_number])))
x = np.concatenate(x, axis=0)
x = np.reshape(x, (num_graphycs*num_samples,point_number*2))
# Тип данных  в нейросетях - float32. Можно и float64, но тогда модель нужно делать с таким типом данных.
#x = x.astype('float32')
x.shape

(30, 4000)

In [8]:
xy = np.concatenate((y,x), axis=1)
xy[0,:]

array([   0.      , -182.154261, -182.813766, ...,  -46.86475 ,
        -44.582413,  -42.288928])

In [9]:
xy = sorted(xy, key=lambda x: random.random())

In [10]:
xy = np.asarray(xy)
xy.shape

(30, 4001)

In [11]:
x, y = xy[:,1:], xy[:,0]
x = x.astype('float32')
y = y.astype('int64')
x.shape, y.shape

((30, 4000), (30,))

In [12]:
# Индексация классов начиается с нуля. То есть первый класс - 0, второй класс - 1. Тип данных разметки - int64.
# Размерность у разметки должна быть одна

In [13]:
# Тут и далее уже очень много ошибок, покажу просто как правильно с комментариями.

class ShaftDataset(Dataset):
    def __init__(self, x, y, mode):
        # Проверка выборки
        if mode == 'train':
            self.x = x[:21]
            self.y = y[:21]
        elif mode == 'val':
            self.x = x[21:]
            self.y = y[21:]
            
        # Разбиение сделано по вашему примеру, но обращаю внимание,
        # что так в тренировочной выборке всего один пример со 2 классом. А в тестовой - только 2 класс/
        # Поэтому нужно как то примеры скомпоновать по-другому

    def __getitem__(self, index):
        return self.x[index], self.y[index]
  
    def __len__(self):
        return len(self.y)

In [14]:
# Создаем два отдельных датасета и их будем подавать в Loader.
# Изначально вы пытались сделать два среза по одному датасету и подавили в Loader. Так работать не будет

train_dataset = ShaftDataset(x, y, 'train')
test_dataset = ShaftDataset(x, y, 'val')


In [15]:
train_dataset[0]

(array([-104.00586, -106.41231, -108.77279, ..., -157.46452, -157.0421 ,
        -156.54283], dtype=float32), 0)

Hyper parameters

In [16]:
input_size = x.shape[1]
hidden_size = 100 # Попробуйте для начала поменьше
num_classes = 2
num_epochs = 100
batch_size = 10 # Bath size не может быть больше размера датасета (у вас 30 примеров)
lr = 0.001 # learning rate

In [17]:
# Loader генерирует тензоры следующего размера -
# (batch_size (размер батча - кол-во примеров в батче), m (кол-во признаков))
# Таким образом, размерностей 2, а не 4

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [18]:
# Размеры данных в датасетах

features, labels = train_dataset.x, train_dataset.y
print(features.shape, labels.shape)
features, labels = test_dataset.x, test_dataset.y
print(features.shape, labels.shape)
#samples = torch.from_numpy(samples)
#labels = torch.from_numpy(labels)

(21, 4000) (21,)
(9, 4000) (9,)


#Создание нейронной сети

In [19]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out


In [20]:
model = NeuralNet(input_size, hidden_size, num_classes)

# Модель также перемещаем в нужное место
model = model.to(device)

Loss and optimizer

In [21]:
L = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Training loop

In [22]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (features, labels) in enumerate(train_loader):
    features = features.to(device)
    labels = labels.to(device)

    #forward path
    outputs = model(features) # Подаем признаки на вход нейросети
    loss = L(outputs, labels)

    #backward path
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 5 == 0:
       print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')


In [23]:
outputs, labels

(tensor([[-15.9476,  10.9707]], grad_fn=<AddmmBackward0>), tensor([1]))

#Test

In [24]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for features, labels in test_loader:
    features = features.to(device)
    labels = labels.to(device)
    outputs = model(features)
    print(outputs)
    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

tensor([[-5.1608e-02,  1.2949e-01],
        [ 1.7375e+01, -2.0112e+01],
        [-5.1608e-02,  1.2949e-01],
        [-5.1608e-02,  1.2949e-01],
        [ 1.0402e+02, -5.1408e+01],
        [-5.1608e-02,  1.2949e-01],
        [ 2.3654e+02, -1.0311e+02],
        [-2.2526e+01,  1.6167e+01],
        [-5.1608e-02,  1.2949e-01]])


In [25]:
  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc} %')

accuracy = 66.66666666666667 %
